<div style="font-size: 20pt; font-weight:bold; justify-content: center;">ETL - Analysis of preventive measures (COVID-19) 1.1.2020. - 2.6.2020. </div>

<div style="font-size: 15pt; font-weight:bold; text-align: center;">ETL - Analiza preventivnih mjera (COVID-19) 1.1.2020. - 2.6.2020. </div>

Part of the project for the _Data Warehouses and Business Intelligence_ course at _Faculty of Organisation and Informatics (FOI), Univesity of Zagreb_.

Ovaj dokument sadrži ETL proces za izgradnju skladišta podataka koje će sadržavati podatke za analizu preventivnih mjera poduzetivh za vrijeme COVID-19 pandemije (u razdoblju 1.1 - 2.6.2020.)

Podaci su preuzeti iz nekolicine izvora:

## Reading into DF

In [1]:
import pandas as pd

In [19]:
df_measures_countries = pd.read_excel("data/measures_countries.xlsx", sheet_name="Database")

In [23]:
df_income_level_countries = pd.read_excel("data/income_level_countries.xlsx", sheet_name="Sheet1")

In [26]:
df_cases_by_country = pd.read_excel("data/cases_by_country.xlsx", sheet_name="COVID-19-geographic-disbtributi")

In [136]:
df_measure_stringency_indexes = pd.read_csv("data/measure_stringency_indexes.csv")

Viewing data.

In [20]:
df_measures_countries.head()

,COUNTRY,ISO,REGION,LOG_TYPE,CATEGORY,MEASURE,TARGETED_POP_GROUP,COMMENTS,NON_COMPLIANCE,DATE_IMPLEMENTED,SOURCE,SOURCE_TYPE,LINK,ENTRY_DATE,Alternative source
0,Netherlands,NLD,Europe,Phase-out measure,Social distancing,Schools closure,Yes,CHECK AGAIN CLOSER TO DATE: Secondary schools ...,Not applicable,2020-06-02,NL Government,Government,https://www.government.nl/topics/coronavirus-c...,2020-04-24,NaN
1,Czech Republic,CZE,Europe,Phase-out measure,Social distancing,Schools closure,Yes,Possibility of attendance at primary schools (...,Not applicable,2020-05-25,Government,Government,https://www.vlada.cz/en/media-centrum/aktualne...,2020-04-17,NaN
2,Czech Republic,CZE,Europe,Phase-out measure,Social distancing,Public services closure,Yes,"Restaurants, accommodation services, taxi serv...",Not applicable,2020-05-25,Government,Government,https://www.vlada.cz/en/media-centrum/aktualne...,2020-04-26,NaN
3,Czech Republic,CZE,Europe,Phase-out measure,Social distancing,Public services closure,Yes,"Theatres, cultural and sporting events (limit ...",Not applicable,2020-05-25,Government,Government,https://www.vlada.cz/en/media-centrum/aktualne...,2020-04-26,NaN
4,Luxembourg,LUX,Europe,Phase-out measure,Social distancing,Schools closure,Yes,Primary schools and Kindergartens resumed; und...,Not applicable,2020-05-25,Government,Government,https://gouvernement.lu/de/actualites/toutes_a...,2020-04-18,NaN


In [1610]:
df_measures_countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7823 entries, 0 to 7822
Data columns (total 15 columns):
country               7823 non-null object
iso                   7823 non-null object
region                7821 non-null object
log_type              7823 non-null object
category              7820 non-null object
measure               7823 non-null object
targeted_pop_group    7823 non-null object
comments              7683 non-null object
non_compliance        6612 non-null object
date_implemented      7601 non-null datetime64[ns]
source                7807 non-null object
source_type           7810 non-null object
link                  7792 non-null object
entry_date            7823 non-null datetime64[ns]
alternative_source    712 non-null object
dtypes: datetime64[ns](2), object(13)
memory usage: 916.9+ KB


In [24]:
df_income_level_countries.head()

,Country Code,Economy,Income Group 2020
0,AFG,Afghanistan,Low income (L)
1,ALB,Albania,Upper middle income (UM)
2,DZA,Algeria,Upper middle income (UM)
3,ASM,American Samoa,Upper middle income (UM)
4,AND,Andorra,High income (H)


In [27]:
df_cases_by_country.head()

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2018,continentExp
0,2020-06-02,2,6,2020,545,8,Afghanistan,AF,AFG,37172386.0,Asia
1,2020-06-01,1,6,2020,680,8,Afghanistan,AF,AFG,37172386.0,Asia
2,2020-05-31,31,5,2020,866,3,Afghanistan,AF,AFG,37172386.0,Asia
3,2020-05-30,30,5,2020,623,11,Afghanistan,AF,AFG,37172386.0,Asia
4,2020-05-29,29,5,2020,580,8,Afghanistan,AF,AFG,37172386.0,Asia


In [1611]:
df_measure_stringency_indexes.head()

,country_name,country_code,date,stringency_index,government_response_index,containment_health_index,economic_support_index
0,Aruba,ABW,2020-01-01,0.0,0.0,0.0,0.0
1,Aruba,ABW,2020-01-02,0.0,0.0,0.0,0.0
2,Aruba,ABW,2020-01-03,0.0,0.0,0.0,0.0
3,Aruba,ABW,2020-01-04,0.0,0.0,0.0,0.0
4,Aruba,ABW,2020-01-05,0.0,0.0,0.0,0.0


## Cleaning the data

### Dropping columns from Stringency index dataset

In [58]:
df_measure_stringency_indexes.head()

,CountryName,CountryCode,Date,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,C3_Cancel public events,C3_Flag,C4_Restrictions on gatherings,...,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
0,Aruba,ABW,20200101,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,ABW,20200102,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,ABW,20200103,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,ABW,20200104,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,ABW,20200105,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [137]:
df_measure_stringency_indexes.columns

Index(['CountryName', 'CountryCode', 'Date', 'C1_School closing', 'C1_Flag',
       'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events', 'C3_Flag',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C5_Close public transport',
       'C5_Flag', 'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'M1_Wildcard', 'ConfirmedCases', 'ConfirmedDeaths', 'StringencyIndex',
       'StringencyIndexForDisplay', 'StringencyLegacyIndex',
       'StringencyLegacyIndexForDisplay', 'GovernmentResponseIndex',
       'GovernmentResponseIndexForDisplay', 'ContainmentHealthIndex',
       'ContainmentHealthIn

In [138]:
cols = df_measure_stringency_indexes.columns[3:-10]

In [61]:
cols

Index(['C1_School closing', 'C1_Flag', 'C2_Workplace closing', 'C2_Flag',
       'C3_Cancel public events', 'C3_Flag', 'C4_Restrictions on gatherings',
       'C4_Flag', 'C5_Close public transport', 'C5_Flag',
       'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'M1_Wildcard', 'ConfirmedCases', 'ConfirmedDeaths'],
      dtype='object')

In [139]:
df_measure_stringency_indexes = df_measure_stringency_indexes.drop(columns=cols)

In [235]:
df_measure_stringency_indexes.columns

Index(['CountryName', 'CountryCode', 'Date', 'StringencyIndex',
       'StringencyIndexForDisplay', 'StringencyLegacyIndex',
       'StringencyLegacyIndexForDisplay', 'GovernmentResponseIndex',
       'GovernmentResponseIndexForDisplay', 'ContainmentHealthIndex',
       'ContainmentHealthIndexForDisplay', 'EconomicSupportIndex',
       'EconomicSupportIndexForDisplay'],
      dtype='object')

In [237]:
df_measure_stringency_indexes = df_measure_stringency_indexes[["CountryName", "CountryCode", "Date", "StringencyIndexForDisplay",
                                         "GovernmentResponseIndexForDisplay", "ContainmentHealthIndexForDisplay",
                                         "EconomicSupportIndexForDisplay"]]

In [239]:
df_measure_stringency_indexes.columns = ["country_name", "country_code", "date", "stringency_index",
                                         "government_response_index", "containment_health_index",
                                         "economic_support_index"]

In [240]:
df_measure_stringency_indexes.head()

,country_name,country_code,date,stringency_index,government_response_index,containment_health_index,economic_support_index
0,Aruba,ABW,2020-01-01,0.0,0.0,0.0,0.0
1,Aruba,ABW,2020-01-02,0.0,0.0,0.0,0.0
2,Aruba,ABW,2020-01-03,0.0,0.0,0.0,0.0
3,Aruba,ABW,2020-01-04,0.0,0.0,0.0,0.0
4,Aruba,ABW,2020-01-05,0.0,0.0,0.0,0.0


### Changing data types in Stringency index dataset

In [140]:
df_measure_stringency_indexes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15994 entries, 0 to 15993
Data columns (total 13 columns):
CountryName                          15994 non-null object
CountryCode                          15994 non-null object
Date                                 15994 non-null int64
StringencyIndex                      15505 non-null float64
StringencyIndexForDisplay            15773 non-null float64
StringencyLegacyIndex                15508 non-null float64
StringencyLegacyIndexForDisplay      15773 non-null float64
GovernmentResponseIndex              15496 non-null float64
GovernmentResponseIndexForDisplay    15769 non-null float64
ContainmentHealthIndex               15501 non-null float64
ContainmentHealthIndexForDisplay     15773 non-null float64
EconomicSupportIndex                 15473 non-null float64
EconomicSupportIndexForDisplay       15740 non-null float64
dtypes: float64(10), int64(1), object(2)
memory usage: 1.6+ MB


In [153]:
df_measure_stringency_indexes["Date"] = pd.to_datetime(df_measure_stringency_indexes["Date"], format="%Y%m%d", errors='coerce')

### Checking date format of all tables

In [159]:
df_measure_stringency_indexes["Date"].head()

0   2020-01-01
1   2020-01-02
2   2020-01-03
3   2020-01-04
4   2020-01-05
Name: Date, dtype: datetime64[ns]

Korištenje QGrid okvira za lakše i brže pregledavanje podataka.

In [160]:
df_cases_by_country["dateRep"].head()

0   2020-06-02
1   2020-06-01
2   2020-05-31
3   2020-05-30
4   2020-05-29
Name: dateRep, dtype: datetime64[ns]

In [162]:
df_measures_countries["DATE_IMPLEMENTED"].head()

0   2020-06-02
1   2020-05-25
2   2020-05-25
3   2020-05-25
4   2020-05-25
Name: DATE_IMPLEMENTED, dtype: datetime64[ns]

### Formatting column names

Column format for all tables will be:
* lowercase
* dunders instead spaces

#### Formatting cases_by_country

In [207]:
df_cases_by_country.head()

,date_rep,day,month,year,cases,deaths,countries_and_territories,geo_id,country_territory_code,pop_data_2018,contient
0,2020-06-02,2,6,2020,545,8,Afghanistan,AF,AFG,37172386.0,Asia
1,2020-06-01,1,6,2020,680,8,Afghanistan,AF,AFG,37172386.0,Asia
2,2020-05-31,31,5,2020,866,3,Afghanistan,AF,AFG,37172386.0,Asia
3,2020-05-30,30,5,2020,623,11,Afghanistan,AF,AFG,37172386.0,Asia
4,2020-05-29,29,5,2020,580,8,Afghanistan,AF,AFG,37172386.0,Asia


In [194]:
new_col_names = []

for c in df_cases_by_country.columns:
    for i,letter in enumerate(c):
        if letter.isupper() or letter.isdigit():
            c = c[:i] + "_" + c[i:]
    c = c.lower()
    new_col_names.append(c)

In [195]:
new_col_names

['date_rep',
 'day',
 'month',
 'year',
 'cases',
 'deaths',
 'countries_an_dterritories',
 'geo_id',
 'countryterritory_code',
 'pop_dat____a2018',
 'continent_exp']

In [196]:
new_col_names[6] = "countries_and_territories"
new_col_names[8] = "country_territory_code"
new_col_names[9] = "pop_data_2018"
new_col_names[10] = "contient"

In [198]:
new_col_names

['date_rep',
 'day',
 'month',
 'year',
 'cases',
 'deaths',
 'countries_and_territories',
 'geo_id',
 'country_territory_code',
 'pop_data_2018',
 'contient']

In [204]:
df_cases_by_country.columns = new_col_names

In [210]:
df_cases_by_country.head()

,date_rep,day,month,year,cases,deaths,countries_and_territories,geo_id,country_territory_code,pop_data_2018,contient
0,2020-06-02,2,6,2020,545,8,Afghanistan,AF,AFG,37172386.0,Asia
1,2020-06-01,1,6,2020,680,8,Afghanistan,AF,AFG,37172386.0,Asia
2,2020-05-31,31,5,2020,866,3,Afghanistan,AF,AFG,37172386.0,Asia
3,2020-05-30,30,5,2020,623,11,Afghanistan,AF,AFG,37172386.0,Asia
4,2020-05-29,29,5,2020,580,8,Afghanistan,AF,AFG,37172386.0,Asia


#### Formatting income_level_countries

In [227]:
df_income_level_countries.head()

,Country Code,Economy,Income Group 2020
0,AFG,Afghanistan,Low income (L)
1,ALB,Albania,Upper middle income (UM)
2,DZA,Algeria,Upper middle income (UM)
3,ASM,American Samoa,Upper middle income (UM)
4,AND,Andorra,High income (H)


In [228]:
new_col_names = []

for c in df_income_level_countries.columns:
    c = c.replace(" ","_").lower()
    new_col_names.append(c)

In [229]:
df_income_level_countries.columns = new_col_names

#### Formatting measures_countries

In [243]:
df_measures_countries.columns

Index(['COUNTRY', 'ISO', 'REGION', 'LOG_TYPE', 'CATEGORY', 'MEASURE',
       'TARGETED_POP_GROUP', 'COMMENTS', 'NON_COMPLIANCE', 'DATE_IMPLEMENTED',
       'SOURCE', 'SOURCE_TYPE', 'LINK', 'ENTRY_DATE', 'Alternative source'],
      dtype='object')

In [245]:
new_col_names = df_measures_countries.columns

new_col_names = [col.lower().replace(" ", "_") for col in new_col_names]

new_col_names

['country',
 'iso',
 'region',
 'log_type',
 'category',
 'measure',
 'targeted_pop_group',
 'comments',
 'non_compliance',
 'date_implemented',
 'source',
 'source_type',
 'link',
 'entry_date',
 'alternative_source']

In [248]:
df_measures_countries.columns = new_col_names

In [249]:
df_measures_countries.head()

,country,iso,region,log_type,category,measure,targeted_pop_group,comments,non_compliance,date_implemented,source,source_type,link,entry_date,alternative_source
0,Netherlands,NLD,Europe,Phase-out measure,Social distancing,Schools closure,Yes,CHECK AGAIN CLOSER TO DATE: Secondary schools ...,Not applicable,2020-06-02,NL Government,Government,https://www.government.nl/topics/coronavirus-c...,2020-04-24,NaN
1,Czech Republic,CZE,Europe,Phase-out measure,Social distancing,Schools closure,Yes,Possibility of attendance at primary schools (...,Not applicable,2020-05-25,Government,Government,https://www.vlada.cz/en/media-centrum/aktualne...,2020-04-17,NaN
2,Czech Republic,CZE,Europe,Phase-out measure,Social distancing,Public services closure,Yes,"Restaurants, accommodation services, taxi serv...",Not applicable,2020-05-25,Government,Government,https://www.vlada.cz/en/media-centrum/aktualne...,2020-04-26,NaN
3,Czech Republic,CZE,Europe,Phase-out measure,Social distancing,Public services closure,Yes,"Theatres, cultural and sporting events (limit ...",Not applicable,2020-05-25,Government,Government,https://www.vlada.cz/en/media-centrum/aktualne...,2020-04-26,NaN
4,Luxembourg,LUX,Europe,Phase-out measure,Social distancing,Schools closure,Yes,Primary schools and Kindergartens resumed; und...,Not applicable,2020-05-25,Government,Government,https://gouvernement.lu/de/actualites/toutes_a...,2020-04-18,NaN


## Merging data

Merging all 4 seperate dataframes into one big dataframe that will become DW Fact tabel - __measure_day_fact__

### Merging _cases_by_country_ to _income_level_countries_

First merging _cases_by_country_ with _income_level_countries_ on ___country_territory_code___ == ___country_code___

In [1102]:
df_cases_by_country.head()

,date_rep,day,month,year,cases,deaths,countries_and_territories,geo_id,country_territory_code,pop_data_2018,contient
0,2020-06-02,2,6,2020,545,8,Afghanistan,AF,AFG,37172386.0,Asia
1,2020-06-01,1,6,2020,680,8,Afghanistan,AF,AFG,37172386.0,Asia
2,2020-05-31,31,5,2020,866,3,Afghanistan,AF,AFG,37172386.0,Asia
3,2020-05-30,30,5,2020,623,11,Afghanistan,AF,AFG,37172386.0,Asia
4,2020-05-29,29,5,2020,580,8,Afghanistan,AF,AFG,37172386.0,Asia


In [1103]:
df_income_level_countries.head()

,country_code,economy,income_group_2020
0,AFG,Afghanistan,Low income (L)
1,ALB,Albania,Upper middle income (UM)
2,DZA,Algeria,Upper middle income (UM)
3,ASM,American Samoa,Upper middle income (UM)
4,AND,Andorra,High income (H)


In [1104]:
df_measure_day_fact = df_cases_by_country.merge(df_income_level_countries[["country_code", "income_group_2020"]], 
                                                how='outer', 
                                                left_on='country_territory_code', right_on='country_code')

In [1114]:
df_measure_day_fact = df_measure_day_fact.drop(columns=["country_territory_code"])

In [1115]:
df_measure_day_fact.head()

,date_rep,day,month,year,cases,deaths,countries_and_territories,geo_id,pop_data_2018,contient,country_code,income_group_2020
0,2020-06-02,2.0,6.0,2020.0,545.0,8.0,Afghanistan,AF,37172386.0,Asia,AFG,Low income (L)
1,2020-06-01,1.0,6.0,2020.0,680.0,8.0,Afghanistan,AF,37172386.0,Asia,AFG,Low income (L)
2,2020-05-31,31.0,5.0,2020.0,866.0,3.0,Afghanistan,AF,37172386.0,Asia,AFG,Low income (L)
3,2020-05-30,30.0,5.0,2020.0,623.0,11.0,Afghanistan,AF,37172386.0,Asia,AFG,Low income (L)
4,2020-05-29,29.0,5.0,2020.0,580.0,8.0,Afghanistan,AF,37172386.0,Asia,AFG,Low income (L)


### Merging _measures_countries_ to _measure_day_fact_ table

Second merger. Merging _measures_countries_ to the _fact_table_ on ___date_rep, country_code___ == ___date_implemented, iso___

In [1116]:
df_measures_countries.head()

,country,iso,region,log_type,category,measure,targeted_pop_group,comments,non_compliance,date_implemented,source,source_type,link,entry_date,alternative_source
0,Netherlands,NLD,Europe,Phase-out measure,Social distancing,Schools closure,Yes,CHECK AGAIN CLOSER TO DATE: Secondary schools ...,Not applicable,2020-06-02,NL Government,Government,https://www.government.nl/topics/coronavirus-c...,2020-04-24,NaN
1,Czech Republic,CZE,Europe,Phase-out measure,Social distancing,Schools closure,Yes,Possibility of attendance at primary schools (...,Not applicable,2020-05-25,Government,Government,https://www.vlada.cz/en/media-centrum/aktualne...,2020-04-17,NaN
2,Czech Republic,CZE,Europe,Phase-out measure,Social distancing,Public services closure,Yes,"Restaurants, accommodation services, taxi serv...",Not applicable,2020-05-25,Government,Government,https://www.vlada.cz/en/media-centrum/aktualne...,2020-04-26,NaN
3,Czech Republic,CZE,Europe,Phase-out measure,Social distancing,Public services closure,Yes,"Theatres, cultural and sporting events (limit ...",Not applicable,2020-05-25,Government,Government,https://www.vlada.cz/en/media-centrum/aktualne...,2020-04-26,NaN
4,Luxembourg,LUX,Europe,Phase-out measure,Social distancing,Schools closure,Yes,Primary schools and Kindergartens resumed; und...,Not applicable,2020-05-25,Government,Government,https://gouvernement.lu/de/actualites/toutes_a...,2020-04-18,NaN


In [1124]:
df_measure_day_fact = df_measure_day_fact.merge(df_measures_countries[["iso","region", "log_type", "category", 
                                                                          "measure", "targeted_pop_group", "comments", 
                                                                          "non_compliance", "date_implemented", "source", 
                                                                          "source_type", "link", "entry_date", 
                                                                          "alternative_source"]], 
                                                  how='outer', 
                                                  left_on=["date_rep", "country_code"], 
                                                  right_on=["date_implemented", "iso"])

In [1127]:
df_measure_day_fact = df_measure_day_fact.drop(columns=["iso"])

In [1132]:
df_measure_day_fact.head()

,date_rep,day,month,year,cases,deaths,countries_and_territories,geo_id,pop_data_2018,contient,...,measure,targeted_pop_group,comments,non_compliance,date_implemented,source,source_type,link,entry_date,alternative_source
0,2020-06-02,2.0,6.0,2020.0,545.0,8.0,Afghanistan,AF,37172386.0,Asia,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN
1,2020-06-01,1.0,6.0,2020.0,680.0,8.0,Afghanistan,AF,37172386.0,Asia,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN
2,2020-05-31,31.0,5.0,2020.0,866.0,3.0,Afghanistan,AF,37172386.0,Asia,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN
3,2020-05-30,30.0,5.0,2020.0,623.0,11.0,Afghanistan,AF,37172386.0,Asia,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN
4,2020-05-29,29.0,5.0,2020.0,580.0,8.0,Afghanistan,AF,37172386.0,Asia,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN


### Merging _stringency_indexes_ to _measure_day_fact_ table 

Final merger. Merging _measure_stringency_indexes_ to the _fact_table_ on ___date___ == ___date_rep___

In [1133]:
df_measure_stringency_indexes.head()

,country_name,country_code,date,stringency_index,government_response_index,containment_health_index,economic_support_index
0,Aruba,ABW,2020-01-01,0.0,0.0,0.0,0.0
1,Aruba,ABW,2020-01-02,0.0,0.0,0.0,0.0
2,Aruba,ABW,2020-01-03,0.0,0.0,0.0,0.0
3,Aruba,ABW,2020-01-04,0.0,0.0,0.0,0.0
4,Aruba,ABW,2020-01-05,0.0,0.0,0.0,0.0


In [1134]:
df_measure_day_fact = df_measure_day_fact.merge(df_measure_stringency_indexes[["country_code", "date", "stringency_index", 
                                                                               "government_response_index", 
                                                                               "containment_health_index", 
                                                                               "economic_support_index"]], 
                                                          how="outer",
                                                          left_on=["date_rep","country_code"],
                                                          right_on=["date", "country_code"])

In [1143]:
df_measure_day_fact = df_measure_day_fact.drop(columns=["date", "date_implemented"])

In [1144]:
df_measure_day_fact.head()

,date_rep,day,month,year,cases,deaths,countries_and_territories,geo_id,pop_data_2018,contient,...,non_compliance,source,source_type,link,entry_date,alternative_source,stringency_index,government_response_index,containment_health_index,economic_support_index
0,2020-06-02,2.0,6.0,2020.0,545.0,8.0,Afghanistan,AF,37172386.0,Asia,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
1,2020-06-01,1.0,6.0,2020.0,680.0,8.0,Afghanistan,AF,37172386.0,Asia,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
2,2020-05-31,31.0,5.0,2020.0,866.0,3.0,Afghanistan,AF,37172386.0,Asia,...,NaN,NaN,NaN,NaN,NaT,NaN,78.7,60.9,71.97,0.0
3,2020-05-30,30.0,5.0,2020.0,623.0,11.0,Afghanistan,AF,37172386.0,Asia,...,NaN,NaN,NaN,NaN,NaT,NaN,78.7,60.9,71.97,0.0
4,2020-05-29,29.0,5.0,2020.0,580.0,8.0,Afghanistan,AF,37172386.0,Asia,...,NaN,NaN,NaN,NaN,NaT,NaN,78.7,60.9,71.97,0.0


## Split data into dimension tables and fact table

This data warehouse will contain of 4 dimension tables and 1 fact table. Next step is to split data from the fact table into dimension tables (where that is necessary) and add foreign keys to fact table. This will be done with _pandas DataFrames_ and in the end these DFs will be exported as _csv_ files. After that, _csv_ files will be read into PostgreSQL data warehouse for storage. 

Dimension tables are:
* Date (dim)
* Country (dim)
* Measure (dim)
* Additional data (junk dim)

In [1180]:
df_date = pd.DataFrame()

In [684]:
df_country = pd.DataFrame()

In [1167]:
df_measure = pd.DataFrame()

In [1204]:
df_additional = pd.DataFrame()

### Setting data into _country_ dimension

In [1146]:
df_measure_day_fact.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30810 entries, 0 to 30809
Data columns (total 28 columns):
date_rep                     24479 non-null datetime64[ns]
day                          24479 non-null float64
month                        24479 non-null float64
year                         24479 non-null float64
cases                        24479 non-null float64
deaths                       24479 non-null float64
countries_and_territories    24479 non-null object
geo_id                       24372 non-null object
pop_data_2018                24169 non-null float64
contient                     24479 non-null object
country_code                 28866 non-null object
income_group_2020            23911 non-null object
region                       7821 non-null object
log_type                     7823 non-null object
category                     7820 non-null object
measure                      7823 non-null object
targeted_pop_group           7823 non-null object
comments       

In [1277]:
country_cols = ["country_code", "region", 
                "countries_and_territories", "geo_id", "pop_data_2018", "contient", "income_group_2020"]
df_country = df_measure_day_fact[country_cols]

In [1278]:
df_country.head()

,country_code,region,countries_and_territories,geo_id,pop_data_2018,contient,income_group_2020
0,AFG,NaN,Afghanistan,AF,37172386.0,Asia,Low income (L)
1,AFG,NaN,Afghanistan,AF,37172386.0,Asia,Low income (L)
2,AFG,NaN,Afghanistan,AF,37172386.0,Asia,Low income (L)
3,AFG,NaN,Afghanistan,AF,37172386.0,Asia,Low income (L)
4,AFG,NaN,Afghanistan,AF,37172386.0,Asia,Low income (L)


In [1279]:
df_country = df_country.drop_duplicates()

In [1280]:
df_country["countries_and_territories"] = df_country["countries_and_territories"].str.replace("_", " ")

In [1281]:
df_country.head()

,country_code,region,countries_and_territories,geo_id,pop_data_2018,contient,income_group_2020
0,AFG,NaN,Afghanistan,AF,37172386.0,Asia,Low income (L)
56,AFG,Asia,Afghanistan,AF,37172386.0,Asia,Low income (L)
165,AFG,NaN,NaN,NaN,NaN,NaN,NaN
175,ALB,NaN,Albania,AL,2866376.0,Europe,Upper middle income (UM)
220,ALB,Europe,Albania,AL,2866376.0,Europe,Upper middle income (UM)


In [1282]:
df_country = df_country.groupby(["country_code","countries_and_territories","income_group_2020", 
                                 "contient", "pop_data_2018", "geo_id", "region"]).sum().reset_index()

In [1283]:
df_country.head()

,country_code,countries_and_territories,income_group_2020,contient,pop_data_2018,geo_id,region
0,AFG,Afghanistan,Low income (L),Asia,37172386.0,AF,Asia
1,AGO,Angola,Lower middle income (LM),Africa,30809762.0,AO,Africa
2,ALB,Albania,Upper middle income (UM),Europe,2866376.0,AL,Europe
3,ARE,United Arab Emirates,High income (H),Asia,9630959.0,AE,Middle East
4,ARG,Argentina,Upper middle income (UM),America,44494502.0,AR,Americas


In [1284]:
df_country["country_id"] = df_country.index

In [1285]:
df_country = df_country[["country_id","countries_and_territories", "country_code", "income_group_2020", 
                         "contient", "pop_data_2018", "geo_id", "region"]]

In [1286]:
df_country.columns = ["country_id","country_name", "country_code", "income_group_2020", 
                      "continent", "pop_data_2018", "gro_id", "region"]

In [1287]:
df_country.head()

,country_id,country_name,country_code,income_group_2020,continent,pop_data_2018,gro_id,region
0,0,Afghanistan,AFG,Low income (L),Asia,37172386.0,AF,Asia
1,1,Angola,AGO,Lower middle income (LM),Africa,30809762.0,AO,Africa
2,2,Albania,ALB,Upper middle income (UM),Europe,2866376.0,AL,Europe
3,3,United Arab Emirates,ARE,High income (H),Asia,9630959.0,AE,Middle East
4,4,Argentina,ARG,Upper middle income (UM),America,44494502.0,AR,Americas


### Setting data into _measures_ dimension

In [1168]:
df_measure_day_fact.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30810 entries, 0 to 30809
Data columns (total 28 columns):
date_rep                     24479 non-null datetime64[ns]
day                          24479 non-null float64
month                        24479 non-null float64
year                         24479 non-null float64
cases                        24479 non-null float64
deaths                       24479 non-null float64
countries_and_territories    24479 non-null object
geo_id                       24372 non-null object
pop_data_2018                24169 non-null float64
contient                     24479 non-null object
country_code                 28866 non-null object
income_group_2020            23911 non-null object
region                       7821 non-null object
log_type                     7823 non-null object
category                     7820 non-null object
measure                      7823 non-null object
targeted_pop_group           7823 non-null object
comments       

In [1169]:
measure_cols = ["category", "measure"]
df_measure = df_measure_day_fact[measure_cols]

In [1170]:
df_measure["measure"] = df_measure["measure"].str.lower()

c:\users\marko\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [1171]:
df_measure.nunique()

category     6
measure     35
dtype: int64

In [1172]:
df_measure = df_measure.groupby(measure_cols).sum().reset_index()

In [1173]:
df_measure['measure'] = df_measure['measure'].str.capitalize()

In [1174]:
df_measure["measure_id"] = df_measure.index

In [1175]:
df_measure = df_measure[["measure_id", "measure", "category"]]

In [1176]:
df_measure.columns = ["measure_id", "measure", "measure_category"]

In [1177]:
df_measure.head()

,measure_id,measure,measure_category
0,0,Economic measures,Governance and socio-economic measures
1,1,Emergency administrative structures activated ...,Governance and socio-economic measures
2,2,Limit product imports/exports,Governance and socio-economic measures
3,3,Military deployment,Governance and socio-economic measures
4,4,State of emergency declared,Governance and socio-economic measures


### Setting data into _date_ dimnesion

In [1181]:
df_measure_day_fact.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30810 entries, 0 to 30809
Data columns (total 28 columns):
date_rep                     24479 non-null datetime64[ns]
day                          24479 non-null float64
month                        24479 non-null float64
year                         24479 non-null float64
cases                        24479 non-null float64
deaths                       24479 non-null float64
countries_and_territories    24479 non-null object
geo_id                       24372 non-null object
pop_data_2018                24169 non-null float64
contient                     24479 non-null object
country_code                 28866 non-null object
income_group_2020            23911 non-null object
region                       7821 non-null object
log_type                     7823 non-null object
category                     7820 non-null object
measure                      7823 non-null object
targeted_pop_group           7823 non-null object
comments       

In [1182]:
df_date = df_measure_day_fact["date_rep"]

In [1183]:
df_date = df_date.drop_duplicates()

In [1184]:
df_date = df_date.sort_values().reset_index()

In [1185]:
df_date.drop(columns=["index"], inplace=True)

In [1186]:
df_date["date_rep"].dropna(inplace=True)

In [1187]:
df_date.drop(df_date.tail(1).index,inplace=True)

In [1188]:
df_date.isnull().values.any()

False

In [1189]:
df_date["year"] = pd.DatetimeIndex(df_date['date_rep']).year
df_date["month"] = pd.DatetimeIndex(df_date['date_rep']).month
df_date["day"] = pd.DatetimeIndex(df_date['date_rep']).day
df_date["day_of_the_week"] = pd.DatetimeIndex(df_date['date_rep']).dayofweek

df_date.head()

,date_rep,year,month,day,day_of_the_week
0,2019-12-31,2019,12,31,1
1,2020-01-01,2020,1,1,2
2,2020-01-02,2020,1,2,3
3,2020-01-03,2020,1,3,4
4,2020-01-04,2020,1,4,5


In [1190]:
df_date.loc[df_date["day_of_the_week"]==0, "day_name"] = "Monday"
df_date.loc[df_date["day_of_the_week"]==1, "day_name"] = "Tuesday"
df_date.loc[df_date["day_of_the_week"]==2, "day_name"] = "Wednsday"
df_date.loc[df_date["day_of_the_week"]==3, "day_name"] = "Thursday"
df_date.loc[df_date["day_of_the_week"]==4, "day_name"] = "Friday"
df_date.loc[df_date["day_of_the_week"]==5, "day_name"] = "Saturday"
df_date.loc[df_date["day_of_the_week"]==6, "day_name"] = "Sunday"

df_date.head()

,date_rep,year,month,day,day_of_the_week,day_name
0,2019-12-31,2019,12,31,1,Tuesday
1,2020-01-01,2020,1,1,2,Wednsday
2,2020-01-02,2020,1,2,3,Thursday
3,2020-01-03,2020,1,3,4,Friday
4,2020-01-04,2020,1,4,5,Saturday


In [1191]:
df_date["month_name"] = pd.DatetimeIndex(df_date['date_rep']).month_name()
df_date["month_name_abbr"] = pd.DatetimeIndex(df_date['date_rep']).month_name().str.slice(stop=3)
df_date["week_of_the_year"] = pd.DatetimeIndex(df_date['date_rep']).week

df_date["is_weekend"] = 0
df_date.loc[df_date['day_of_the_week'].isin([5, 6]), 'is_weekend'] = 1

df_date["quarter"] = pd.DatetimeIndex(df_date['date_rep']).quarter

df_date.head()

,date_rep,year,month,day,day_of_the_week,day_name,month_name,month_name_abbr,week_of_the_year,is_weekend,quarter
0,2019-12-31,2019,12,31,1,Tuesday,December,Dec,1,0,4
1,2020-01-01,2020,1,1,2,Wednsday,January,Jan,1,0,1
2,2020-01-02,2020,1,2,3,Thursday,January,Jan,1,0,1
3,2020-01-03,2020,1,3,4,Friday,January,Jan,1,0,1
4,2020-01-04,2020,1,4,5,Saturday,January,Jan,1,1,1


In [1192]:
df_date["date_id"] = df_date.index
df_date.head()

,date_rep,year,month,day,day_of_the_week,day_name,month_name,month_name_abbr,week_of_the_year,is_weekend,quarter,date_id
0,2019-12-31,2019,12,31,1,Tuesday,December,Dec,1,0,4,0
1,2020-01-01,2020,1,1,2,Wednsday,January,Jan,1,0,1,1
2,2020-01-02,2020,1,2,3,Thursday,January,Jan,1,0,1,2
3,2020-01-03,2020,1,3,4,Friday,January,Jan,1,0,1,3
4,2020-01-04,2020,1,4,5,Saturday,January,Jan,1,1,1,4


In [1193]:
date_cols = ['date_id','date_rep', 'year', 'month', 'day', 'day_of_the_week', 'day_name',
       'month_name', 'month_name_abbr', 'week_of_the_year', 'is_weekend',
       'quarter']
df_date = df_date[date_cols]
df_date.head()

,date_id,date_rep,year,month,day,day_of_the_week,day_name,month_name,month_name_abbr,week_of_the_year,is_weekend,quarter
0,0,2019-12-31,2019,12,31,1,Tuesday,December,Dec,1,0,4
1,1,2020-01-01,2020,1,1,2,Wednsday,January,Jan,1,0,1
2,2,2020-01-02,2020,1,2,3,Thursday,January,Jan,1,0,1
3,3,2020-01-03,2020,1,3,4,Friday,January,Jan,1,0,1
4,4,2020-01-04,2020,1,4,5,Saturday,January,Jan,1,1,1


### Setting data into _additional_data_ dimension

In [1205]:
df_measure_day_fact.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30810 entries, 0 to 30809
Data columns (total 28 columns):
date_rep                     24479 non-null datetime64[ns]
day                          24479 non-null float64
month                        24479 non-null float64
year                         24479 non-null float64
cases                        24479 non-null float64
deaths                       24479 non-null float64
countries_and_territories    24479 non-null object
geo_id                       24372 non-null object
pop_data_2018                24169 non-null float64
contient                     24479 non-null object
country_code                 28866 non-null object
income_group_2020            23911 non-null object
region                       7821 non-null object
log_type                     7823 non-null object
category                     7820 non-null object
measure                      7823 non-null object
targeted_pop_group           7823 non-null object
comments       

In [1206]:
additional_cols = ["log_type", "targeted_pop_group", "comments", "non_compliance", "source", "source_type", 
                   "link", "entry_date", "alternative_source"]
df_additional = df_measure_day_fact[additional_cols]

In [1207]:
df_additional.drop_duplicates(inplace=True)

c:\users\marko\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [1208]:
df_additional = df_additional.reset_index()

In [1209]:
df_additional.drop(columns=["index"], inplace=True)

In [1210]:
df_additional["additional_id"] = df_additional.index

In [1212]:
add_cols = ["additional_id", "comments", "non_compliance", "source", "source_type", "entry_date", "alternative_source", 
            "link", "log_type", "targeted_pop_group"]
df_additional = df_additional[add_cols]

In [1213]:
df_additional.head()

,additional_id,comments,non_compliance,source,source_type,entry_date,alternative_source,link,log_type,targeted_pop_group
0,0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1,1,A COVID-19 lab is opened using Government of A...,NaN,Afghanistan MoPH,Government,2020-04-07,https://af.usembassy.gov/covid-19-information/,https://www.facebook.com/af.moph/posts/1060980...,Introduction / extension of measures,No
2,2,Machines for health care were sent to Herat Pr...,NaN,Afghanistan MoPH,Government,2020-04-07,NaN,https://www.facebook.com/af.moph/photos/a.4626...,Introduction / extension of measures,No
3,3,Supplies from China received today,NaN,Afghanistan MoPH,Government,2020-04-07,NaN,https://www.facebook.com/af.moph/posts/1057171...,Introduction / extension of measures,No
4,4,2nd diagnostic lab opened today,NaN,Afghanistan MoPH,Government,2020-04-07,NaN,https://www.facebook.com/af.moph/posts/1055005...,Introduction / extension of measures,No


## Adding foreign keys to _fact_ table

Finally, it is time to remove all unnecessary data from _fact_ table and connect it with dimensions via foreign keys relationships.

### Adding _country_dim_ key

Table containing _country_ data will be connected with the _fact_ table by merging columns _country_name_ and _contries_and_territories_ (just like in merging _country_ dimension) and then setting foreign key via that value.

In [1218]:
#df_measure_day_fact.to_csv("raw_fact2.csv", index=False)

In [1568]:
df_measure_day_fact = pd.read_csv("raw_fact2.csv", index_col=None)

In [1569]:
df_measure_day_fact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30810 entries, 0 to 30809
Data columns (total 28 columns):
date_rep                     24479 non-null object
day                          24479 non-null float64
month                        24479 non-null float64
year                         24479 non-null float64
cases                        24479 non-null float64
deaths                       24479 non-null float64
countries_and_territories    24479 non-null object
geo_id                       24372 non-null object
pop_data_2018                24169 non-null float64
contient                     24479 non-null object
country_code                 28866 non-null object
income_group_2020            23911 non-null object
region                       7821 non-null object
log_type                     7823 non-null object
category                     7820 non-null object
measure                      7823 non-null object
targeted_pop_group           7823 non-null object
comments               

In [1570]:
df_measure_day_fact = df_measure_day_fact[df_measure_day_fact["country_code"].notnull()]

In [1571]:
df_measure_day_fact = df_measure_day_fact.merge(df_country[["country_id", "country_code"]], how='outer', 
                                                            left_on = "country_code", right_on="country_code")

In [1572]:
df_measure_day_fact.drop(columns=["country_code", "countries_and_territories", 
                                  "income_group_2020", "contient", "pop_data_2018", "geo_id"], inplace=True)

In [1573]:
df_measure_day_fact.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28866 entries, 0 to 28865
Data columns (total 23 columns):
date_rep                     23884 non-null object
day                          23884 non-null float64
month                        23884 non-null float64
year                         23884 non-null float64
cases                        23884 non-null float64
deaths                       23884 non-null float64
region                       6472 non-null object
log_type                     6474 non-null object
category                     6472 non-null object
measure                      6474 non-null object
targeted_pop_group           6474 non-null object
comments                     6387 non-null object
non_compliance               5614 non-null object
source                       6469 non-null object
source_type                  6470 non-null object
link                         6461 non-null object
entry_date                   6474 non-null object
alternative_source           6

### Adding _date_dim_ key

In [1574]:
df_measure_day_fact['date_rep'] = pd.to_datetime(df_measure_day_fact['date_rep'], format='%Y-%m-%d')

In [1575]:
df_measure_day_fact.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28866 entries, 0 to 28865
Data columns (total 23 columns):
date_rep                     23884 non-null datetime64[ns]
day                          23884 non-null float64
month                        23884 non-null float64
year                         23884 non-null float64
cases                        23884 non-null float64
deaths                       23884 non-null float64
region                       6472 non-null object
log_type                     6474 non-null object
category                     6472 non-null object
measure                      6474 non-null object
targeted_pop_group           6474 non-null object
comments                     6387 non-null object
non_compliance               5614 non-null object
source                       6469 non-null object
source_type                  6470 non-null object
link                         6461 non-null object
entry_date                   6474 non-null object
alternative_source    

In [1576]:
df_measure_day_fact = df_measure_day_fact.merge(df_date[["date_id", "date_rep"]], 
                                                how='outer', 
                                                left_on="date_rep", 
                                                right_on="date_rep")

In [1577]:
df_measure_day_fact.drop(columns=["date_rep", "year", "month", "day", "entry_date", "region"], inplace=True)

In [1578]:
df_measure_day_fact.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28866 entries, 0 to 28865
Data columns (total 18 columns):
cases                        23884 non-null float64
deaths                       23884 non-null float64
log_type                     6474 non-null object
category                     6472 non-null object
measure                      6474 non-null object
targeted_pop_group           6474 non-null object
comments                     6387 non-null object
non_compliance               5614 non-null object
source                       6469 non-null object
source_type                  6470 non-null object
link                         6461 non-null object
alternative_source           616 non-null object
stringency_index             17832 non-null float64
government_response_index    17828 non-null float64
containment_health_index     17832 non-null float64
economic_support_index       17780 non-null float64
country_id                   25697 non-null float64
date_id                     

### Adding _measure_dim_ key

In [1579]:
df_measure_day_fact = df_measure_day_fact.merge(df_measure[["measure_id", "measure"]], 
                                                how='outer', 
                                                left_on="measure", 
                                                right_on="measure")

In [1580]:
df_measure_day_fact.drop(columns=["measure","category"], inplace=True)

In [1581]:
df_measure_day_fact.head()

,cases,deaths,log_type,targeted_pop_group,comments,non_compliance,source,source_type,link,alternative_source,stringency_index,government_response_index,containment_health_index,economic_support_index,country_id,date_id,measure_id
0,545.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,154.0,NaN
1,6.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,154.0,NaN
2,119.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.0,154.0,NaN
3,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,154.0,NaN
4,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,154.0,NaN


### Adding _additional_dim_ key

In [1582]:
df_additional.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7751 entries, 0 to 7750
Data columns (total 10 columns):
additional_id         7751 non-null int64
comments              7628 non-null object
non_compliance        6559 non-null object
source                7737 non-null object
source_type           7740 non-null object
entry_date            7750 non-null datetime64[ns]
alternative_source    709 non-null object
link                  7722 non-null object
log_type              7750 non-null object
targeted_pop_group    7750 non-null object
dtypes: datetime64[ns](1), int64(1), object(8)
memory usage: 605.7+ KB


In [1583]:
df_measure_day_fact.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28867 entries, 0 to 28866
Data columns (total 17 columns):
cases                        23884 non-null float64
deaths                       23884 non-null float64
log_type                     6474 non-null object
targeted_pop_group           6474 non-null object
comments                     6387 non-null object
non_compliance               5614 non-null object
source                       6469 non-null object
source_type                  6470 non-null object
link                         6461 non-null object
alternative_source           616 non-null object
stringency_index             17832 non-null float64
government_response_index    17828 non-null float64
containment_health_index     17832 non-null float64
economic_support_index       17780 non-null float64
country_id                   25697 non-null float64
date_id                      23884 non-null float64
measure_id                   6444 non-null float64
dtypes: float64(9), objec

In [1584]:
df_measure_day_fact = df_measure_day_fact.merge(df_additional, 
                                                how='outer', 
                                                left_on=["log_type", "targeted_pop_group","comments", 
                                                         "non_compliance", "source", "source_type", 
                                                         "link", "alternative_source"], 
                                                right_on=["log_type", "targeted_pop_group","comments", 
                                                         "non_compliance", "source", "source_type", 
                                                         "link", "alternative_source"])

In [1585]:
df_measure_day_fact.drop(columns=["comments", "log_type", "targeted_pop_group", "non_compliance", 
                                  "source", "source_type", "link", "alternative_source", "entry_date"], inplace=True)

In [1586]:
df_measure_day_fact = df_measure_day_fact[df_measure_day_fact["country_id"].notnull()]
df_measure_day_fact = df_measure_day_fact[df_measure_day_fact["date_id"].notnull()].reset_index()

In [1587]:
df_measure_day_fact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21658 entries, 0 to 21657
Data columns (total 11 columns):
index                        21658 non-null int64
cases                        21658 non-null float64
deaths                       21658 non-null float64
stringency_index             12433 non-null float64
government_response_index    12429 non-null float64
containment_health_index     12433 non-null float64
economic_support_index       12383 non-null float64
country_id                   21658 non-null float64
date_id                      21658 non-null float64
measure_id                   6385 non-null float64
additional_id                21658 non-null int64
dtypes: float64(9), int64(2)
memory usage: 1.8 MB


In [1588]:
df_measure_day_fact = df_measure_day_fact.drop_duplicates()

## Final touches

In [1589]:
df_measure_day_fact.head()

,index,cases,deaths,stringency_index,government_response_index,containment_health_index,economic_support_index,country_id,date_id,measure_id,additional_id
0,0,545.0,8.0,NaN,NaN,NaN,NaN,0.0,154.0,NaN,0
1,1,6.0,0.0,NaN,NaN,NaN,NaN,2.0,154.0,NaN,0
2,2,119.0,8.0,NaN,NaN,NaN,NaN,47.0,154.0,NaN,0
3,4,0.0,0.0,NaN,NaN,NaN,NaN,1.0,154.0,NaN,0
4,5,0.0,0.0,NaN,NaN,NaN,NaN,6.0,154.0,NaN,0


In [1590]:
td = df_measure_day_fact

In [1591]:
df_measure_day_fact.columns

Index(['index', 'cases', 'deaths', 'stringency_index',
       'government_response_index', 'containment_health_index',
       'economic_support_index', 'country_id', 'date_id', 'measure_id',
       'additional_id'],
      dtype='object')

In [1592]:
df_measure_day_fact.head()

,index,cases,deaths,stringency_index,government_response_index,containment_health_index,economic_support_index,country_id,date_id,measure_id,additional_id
0,0,545.0,8.0,NaN,NaN,NaN,NaN,0.0,154.0,NaN,0
1,1,6.0,0.0,NaN,NaN,NaN,NaN,2.0,154.0,NaN,0
2,2,119.0,8.0,NaN,NaN,NaN,NaN,47.0,154.0,NaN,0
3,4,0.0,0.0,NaN,NaN,NaN,NaN,1.0,154.0,NaN,0
4,5,0.0,0.0,NaN,NaN,NaN,NaN,6.0,154.0,NaN,0


In [1593]:
df_measure_day_fact.reset_index(inplace=True)
df_measure_day_fact.drop(columns=["index"],inplace=True)
df_measure_day_fact.drop(columns=["level_0"],inplace=True)
df_measure_day_fact.head()

,cases,deaths,stringency_index,government_response_index,containment_health_index,economic_support_index,country_id,date_id,measure_id,additional_id
0,545.0,8.0,NaN,NaN,NaN,NaN,0.0,154.0,NaN,0
1,6.0,0.0,NaN,NaN,NaN,NaN,2.0,154.0,NaN,0
2,119.0,8.0,NaN,NaN,NaN,NaN,47.0,154.0,NaN,0
3,0.0,0.0,NaN,NaN,NaN,NaN,1.0,154.0,NaN,0
4,0.0,0.0,NaN,NaN,NaN,NaN,6.0,154.0,NaN,0


In [1594]:
df_measure_day_fact["country_id"] = df_measure_day_fact["country_id"].astype(int)
df_measure_day_fact["date_id"] = df_measure_day_fact["date_id"].astype(int)
df_measure_day_fact["cases"] = df_measure_day_fact["cases"].astype(int)
df_measure_day_fact["deaths"] = df_measure_day_fact["deaths"].astype(int)

In [1595]:
df_measure_day_fact.head()

,cases,deaths,stringency_index,government_response_index,containment_health_index,economic_support_index,country_id,date_id,measure_id,additional_id
0,545,8,NaN,NaN,NaN,NaN,0,154,NaN,0
1,6,0,NaN,NaN,NaN,NaN,2,154,NaN,0
2,119,8,NaN,NaN,NaN,NaN,47,154,NaN,0
3,0,0,NaN,NaN,NaN,NaN,1,154,NaN,0
4,0,0,NaN,NaN,NaN,NaN,6,154,NaN,0


In [1596]:
columns_names = ["date_id", "country_id", "measure_id", "additional_id", 
                 "cases", "deaths", "stringency_index", "government_response_index",
                 "containment_health_index", "economic_support_index"]
df_measure_day_fact = df_measure_day_fact[columns_names]
df_measure_day_fact = df_measure_day_fact.sort_values(by=["date_id"])

df_measure_day_fact.reset_index(inplace=True)
df_measure_day_fact.drop(columns=["index"],inplace=True)

df_measure_day_fact.head()

,date_id,country_id,measure_id,additional_id,cases,deaths,stringency_index,government_response_index,containment_health_index,economic_support_index
0,0,48,NaN,0,0,0,NaN,NaN,NaN,NaN
1,0,76,NaN,0,0,0,NaN,NaN,NaN,NaN
2,0,75,NaN,0,0,0,NaN,NaN,NaN,NaN
3,0,72,NaN,0,0,0,NaN,NaN,NaN,NaN
4,0,73,NaN,0,0,0,NaN,NaN,NaN,NaN


### Exporting tables as _csv_

In [1602]:
df_country.to_csv("processedData/country_dim.csv", index=False)

In [1604]:
df_measure.to_csv("processedData/measure_dim.csv", index=False)

In [1606]:
df_date.to_csv("processedData/date_dim.csv", index=False)

In [1608]:
df_additional.to_csv("processedData/additional_dim.csv", index=False)

In [1609]:
df_measure_day_fact.to_csv("processedData/measure_day_fact.csv", index=False)

## Analysing with QGrid

In [1597]:
import qgrid

In [1598]:
qd = qgrid.QgridWidget(df=df_measure_day_fact, show_toolbar=True)

In [1599]:
qd

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…